In [1]:
from blockchain import Blockchain, Block

blockchain = Blockchain()

genesis = blockchain.chain[0]
b = blockchain.generate_new_block()
print(genesis.__dict__, "\n\n")
print(b.__dict__)

{'index': 0, 'hash': '0000607e58623c07e2634999e38638e13b76af5b450cd7a295bc60179e6db208', 'previous_hash': '', 'timestamp': 'inthebeninging', 'proof': 33} 


{'index': 1, 'hash': '0000ef765efc5694224dfd2b35ab6f59213cc641e5fa84b1d609de977d9acae7', 'previous_hash': '0000607e58623c07e2634999e38638e13b76af5b450cd7a295bc60179e6db208', 'timestamp': 1638823369.7528887, 'proof': 36807}


# trying out transactions

In [21]:
from transaction import Transaction, TxIn, TxOut

txin1 = TxIn("a", 1)
txin2 = TxIn("b", 2)
txin3 = TxIn("c", 3)

txins = [txin1, txin2, txin3]

txout1 = TxOut("d", 10)
txout2 = TxOut("g", 15)
txout3 = TxOut("f", 12)

txouts = [txout1, txout2, txout3]

transaction = Transaction(txins, txouts)

transaction.__dict__

{'txIns': [<transaction.TxIn at 0x7f7a5e8a5340>,
 'txOuts': [<transaction.TxOut at 0x7f7a5e8a5880>,
 'id': '43c274a2dbc1325d84cc015dbddcfabe7a4b08b69bda9ec456adb94ce8e04b67'}